<a href="https://colab.research.google.com/github/Hlompho-Dash/first_try_of_fastai/blob/master/03_minibatch_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [65]:
#export
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_02 import *
import torch.nn.functional as F
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

**Data**

In [66]:
mpl.rcParams["image.cmap"] = "gray"

In [67]:
x_train,y_train,x_valid,y_valid = get_data()

In [68]:
n,m = x_train.shape
c = y_train.max() + 1
nh = 50

In [69]:
class Model(nn.Module):
  def __init__(self, n_in, nh, n_out):
    super().__init__()
    self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
  def __call__(self, x):
    for l in self.layers: x = l(x)
    return x

In [70]:
model = Model(m,nh,10)

In [71]:
pred = model(x_train)

**Cross Entropy Loss**

In [72]:
def log_softmax(x): return (x.exp()/(x.exp().sum(-1,keepdim=True))).log()

In [73]:
sm_pred = log_softmax(pred)
sm_pred

tensor([[-2.2677, -2.3908, -2.1650,  ..., -2.2638, -2.3224, -2.2185],
        [-2.2558, -2.3419, -2.2427,  ..., -2.2805, -2.2712, -2.2322],
        [-2.2938, -2.3642, -2.2378,  ..., -2.2178, -2.3266, -2.2225],
        ...,
        [-2.2646, -2.3623, -2.1283,  ..., -2.1977, -2.2917, -2.3566],
        [-2.2970, -2.4061, -2.1789,  ..., -2.2479, -2.3271, -2.2348],
        [-2.2604, -2.4039, -2.0971,  ..., -2.2922, -2.3473, -2.2571]],
       grad_fn=<LogBackward>)

In [74]:
y_train[:3]

tensor([5, 0, 4])

In [75]:
sm_pred[[0,1,2], [5,0,4]]

tensor([-2.2823, -2.2558, -2.3313], grad_fn=<IndexBackward>)

In [76]:
x_train.shape[0]

50000

In [77]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [78]:
loss = nll(sm_pred, y_train)

In [79]:
loss

tensor(2.3043, grad_fn=<NegBackward>)

In [80]:
def log_softmax(x): return x - x.exp().sum(-1,keepdim=True).log()

In [81]:
test_near(nll(log_softmax(pred), y_train), loss)

In [82]:
def logsumexp(x):
  m = x.max(-1)[0]
  return m + (x-m[:,None]).exp().sum(-1).log()

In [83]:
test_near(logsumexp(pred), pred.logsumexp(-1))

In [84]:
def log_softmax(x): return x - x.logsumexp(-1, keepdim = True)

In [85]:
test_near(nll(log_softmax(pred),y_train), loss)

In [86]:
test_near(F.nll_loss(F.log_softmax(pred,-1), y_train), loss)

In [87]:
test_near(F.cross_entropy(pred, y_train), loss)

BASIC TRAINING LOOP

In [91]:
loss_func = F.cross_entropy

In [89]:
#export

def accuracy(out, yb): return (torch.argmax(out, dim = 1) == yb).float().mean()

In [90]:
bs = 64
xb = x_train[0:bs]
preds = model(xb)
preds[0]

tensor([ 0.0081, -0.1150,  0.1108, -0.0554, -0.0728, -0.0065, -0.1934,  0.0120,
        -0.0466,  0.0573], grad_fn=<SelectBackward>)

In [92]:
yb = y_train[0:bs]
loss_func(preds, yb)

tensor(2.2999, grad_fn=<NllLossBackward>)

In [93]:
accuracy(preds, yb)

tensor(0.0938)

In [94]:
lr = 0.5
epochs = 1

In [96]:
for epoch in range(epochs):
  for i in range((n-1)//bs + 1):
    start_i = i*bs
    end_i = start_i*bs

    xb = x_train[start_i:end_i]
    yb = y_train[start_i:end_i]

    loss = loss_func(model(xb), yb)

    loss.backward()
    with torch.no_grad():
      for l in model.layers:
        if hasattr(l, "weight"):
          l.weight -= l.weight.grad * lr
          l.bias -= l.bias.grad * lr

          l.weight.grad.zero_()
          l.bias.grad.zero_()

In [98]:
loss_func(model(xb), yb), accuracy(model(xb), yb)

(tensor(0.0061, grad_fn=<NllLossBackward>), tensor(1.))

**UNSING PARAMEMTERS AND OPTIM**

Parameters